In [1]:
from geopy.distance import geodesic
import os
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point
from typing import List

    It seems like you want to merge your df DataFrame, which contains information about charging stations, with the all_features_gdf GeoDataFrame that contains information about different kinds of amenities for different years. The goal is to enrich your charging station data with information about the surrounding amenities for each year.

    Here is a strategy to do this:

    1. Compute the distance between each charging station and all the amenities for a specific year.
    2. Compute the descriptive statistics (total count, average distance, minimum distance, maximum distance, etc.) for each type of amenity.
    3. Add these statistics as new columns to your charging stations DataFrame (df) for each year.
    
    Let's break it down:

First, define a function to compute distance between two geographical points (coordinates are in (latitude, longitude) format). Note that we'll use geopy library for this:

Next, we need to create a function to compute the distances of a point to all amenities and return the summary statistics:

Now we need to iterate over all rows in the df DataFrame and for each row (charging station), we compute the summary statistics for each amenity in the all_features_gdf GeoDataFrame. Here is the code snippet:

In [2]:
def load_gdfs():
    # Get the current working directory
    cwd = os.getcwd()

    # Define the relative paths to the data files
    all_features_gdf_path = os.path.join(cwd, '..', 'data', 'interim', 'all_features_gdf.csv')
    df_path = os.path.join(cwd, '..', 'data', 'interim', 'train_gdf_forward_geocoded.csv')

    # Load the dataframes
    all_features_df = pd.read_csv(all_features_gdf_path)
    df = pd.read_csv(df_path)

    # Convert 'geometry' column to geometry type
    all_features_df['geometry'] = all_features_df['geometry'].apply(wkt.loads)
    df['geometry'] = df['geometry'].apply(wkt.loads)

    # Convert the pandas DataFrames to GeoDataFrames
    all_features_gdf = gpd.GeoDataFrame(all_features_df, geometry='geometry')
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    return all_features_gdf, gdf

all_features_gdf, gdf = load_gdfs()


In [3]:
all_features_gdf.head()

,Unnamed: 0,geometry,osm_year,fuel_station,parking_station,busstation_station,trainstation_station,mall_station,supermarket_station,restaurant_station,hotel_station,school_station,university_station,cinema_station,theatre_station
0,0,POINT (-4.34409 55.93436),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,POINT (-3.72159 56.69738),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,POINT (-3.16965 56.19748),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,POINT (-3.29724 55.89943),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,POINT (-3.28774 55.92326),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
gdf.head()

,Unnamed: 0.1,Unnamed: 0,CP ID,Connector,Total kWh,Site,Model,End DateTime,Site_encoded,Model_encoded,Total kWh.1,geometry
0,0,0,50994,1,2.084,Leslie Street Car Park,APT Triple Rapid Charger,2016-01-09 07:27:00,0.0,3.0,2.084,POINT (-3.33802 56.59132)
1,1,1,50281,2,3.870,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 09:01:00,19.0,0.0,3.870,POINT (-3.73882 56.70342)
2,2,2,50285,1,13.930,Broxden Park & Ride,APT 22kW Dual Outlet,2016-01-09 14:32:00,3.0,0.0,13.930,POINT (-3.47775 56.38661)
3,3,3,50281,1,10.380,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 16:37:00,19.0,0.0,10.380,POINT (-3.73882 56.70342)
4,4,4,50745,2,3.580,Kinross Park and Ride,APT Triple Rapid Charger,2016-01-09 09:37:00,13.0,3.0,3.580,POINT (-3.43295 56.20673)


In [5]:
def calculate_amenity_distances(gdf: gpd.GeoDataFrame, amenity_gdf: gpd.GeoDataFrame, amenity: str, year: int, index: int) -> pd.DataFrame:
    # Compute distances
    station_coord = gdf.loc[index, 'geometry']
    distances = amenity_gdf['geometry'].apply(
        lambda x: geodesic((station_coord.y, station_coord.x), (x.y, x.x)).km
    )

    # Compute statistics
    distances = distances.astype(float)
    stats = {
        f'{amenity}_count_{year}': len(distances),
        f'{amenity}_avg_distance_{year}': distances.mean(),
        f'{amenity}_min_distance_{year}': distances.min(),
        f'{amenity}_max_distance_{year}': distances.max()
    }

    return pd.DataFrame(stats, index=[index])

In [6]:
# def calculate_amenity_stats(gdf: gpd.GeoDataFrame, all_features_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
#     # Define all types of amenities
#     amenities = ['fuel_station', 'parking_station', 'busstation_station', 'trainstation_station', 
#                     'mall_station', 'supermarket_station', 'restaurant_station', 'hotel_station', 
#                     'school_station', 'university_station', 'cinema_station', 'theatre_station']

#     all_stats = []
#     # Iterate over all rows in the GeoDataFrame
#     for index, _ in gdf.iterrows():
#         # Iterate over all years
#         for year in range(2016, 2020):  # adjust this as per your data
#             # Extract amenities for the year
#             year_amenities_gdf = all_features_gdf[all_features_gdf['osm_year'] == year]

#             # Iterate over all types of amenities
#             for amenity in amenities:
#                 # Extract the GeoDataFrame for the specific amenity
#                 amenity_gdf = year_amenities_gdf.dropna(subset=[amenity])

#                 # Calculate distances and statistics for the amenity
#                 stats = calculate_amenity_distances(gdf, amenity_gdf, amenity, year, index)
#                 all_stats.append(stats)

#     # Concatenate all stats dataframes
#     all_stats_df = pd.concat(all_stats, axis=0)

#     # Join the stats dataframe with the original geodataframe
#     gdf = gdf.join(all_stats_df)

#     return gdf

In [7]:
from joblib import Parallel, delayed

def calculate_row_amenity_stats(index: int, row: pd.Series, all_features_gdf: gpd.GeoDataFrame, amenities: list) -> pd.DataFrame:
    all_stats = []
    # Iterate over all years
    for year in range(2016, 2020):  # adjust this as per your data
        # Extract amenities for the year
        year_amenities_gdf = all_features_gdf[all_features_gdf['osm_year'] == year]

        # Iterate over all types of amenities
        for amenity in amenities:
            # Extract the GeoDataFrame for the specific amenity
            amenity_gdf = year_amenities_gdf.dropna(subset=[amenity])

            # Calculate distances and statistics for the amenity
            stats = calculate_amenity_distances(row.to_frame().T, amenity_gdf, amenity, year, index)
            all_stats.append(stats)

    # Concatenate all stats dataframes
    return pd.concat(all_stats, axis=0)

def calculate_amenity_stats(gdf: gpd.GeoDataFrame, all_features_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    # Define all types of amenities
    amenities = ['fuel_station', 'parking_station', 'busstation_station', 'trainstation_station', 
                    'mall_station', 'supermarket_station', 'restaurant_station', 'hotel_station', 
                    'school_station', 'university_station', 'cinema_station', 'theatre_station']

    # Use joblib to parallelize the calculation
    results = Parallel(n_jobs=-1)(delayed(calculate_row_amenity_stats)(index, row, all_features_gdf, amenities) for index, row in gdf.iterrows())

    # Join the results with the original geodataframe
    gdf = pd.concat([gdf, pd.concat(results, axis=0)], axis=1)

    return gdf

In [9]:
import time

start_time = time.time()

from joblib import parallel_backend

with parallel_backend('loky'):
    gdf = calculate_amenity_stats(gdf, all_features_gdf)

end_time = time.time()

print(f"Total time: {end_time - start_time} seconds")

In [ ]:
# gdf = calculate_amenity_stats(gdf, all_features_gdf)